# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096864


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:05<01:23,  2.88s/it]

Rendering models:  10%|▉         | 3/31 [00:06<00:59,  2.14s/it]

Rendering models:  16%|█▌        | 5/31 [00:06<00:40,  1.57s/it]

Rendering models:  19%|█▉        | 6/31 [00:06<00:28,  1.15s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:15<00:01,  2.69it/s]

Rendering models:  94%|█████████▎| 29/31 [00:15<00:00,  2.84it/s]

Rendering models:  97%|█████████▋| 30/31 [00:16<00:00,  2.85it/s]

Rendering models: 100%|██████████| 31/31 [00:16<00:00,  2.96it/s]

DannyU                                 0.000479
Jonas_Cross                            0.000454
jnarayanbvg                            0.038597
not-logged-in-c6917d9b19f41e6c2995     0.021464
awright5                               0.002217
Haransis                               0.033691
not-logged-in-74261f68b83c7ae82ce3     0.001174
TSM_Fatboy                            25.415944
severing_ties_with_dema                0.051702
not-logged-in-746dafd91ba51764d0ed     0.001853
rudolf_wieczorek                       0.000983
AdamMorse                              0.000804
not-logged-in-73b973978dc7c9e2e5ac     0.051644
AlexYoung35                            0.000595
Lavadude                               0.002216
ShirleyL                               0.000559
clars915                               0.001428
not-logged-in-136cf6b9b18435d130dd     0.000771
jrc04                                  0.001096
nicholas.                              0.003166
nyoung55                               0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())